In [1]:
import pdfplumber

file_path = 'C:\\guardian\\SD%20Payor%20Scraping\\guardian.pdf'      

In [2]:
with pdfplumber.open(file_path) as pdf:
  
    for page in pdf.pages:
        tables = page.extract_tables()
        t = 1
        for table in tables:
            print("\n")
#             print("Table", t)
            t += 1
            r = 1
#             print(len(table))
#             print(table[2])    
            for row in table[:-1]:
#                 print("Row", r)
#                 print(row)
                
                r +=1
                if 'Claim Number' in row[0]:
                    text = row[0]
                    
                if 'Line' in row[0]:
                    header = row
#                     print("Header", header)
                

In [3]:
text

'Claim Number: 36347F13200 Patient Account No.:120320346 Plan Number:00581682\nPatient Name: ANDREW CASEY Employee Name: ANDREW CASEY Relationship: SON\nPlanholder: GOENGINEER, INC.'

In [4]:
# text = 'Claim Number: 18472E05500 Patient Account No.:15320 Plan Number:00580287 Patient Name: HARPER L NEWTON Employee Name: MICHELLE N BERMAN Relationship: DAUGHTER Planholder: GDH CONSULTING, INC.'

In [5]:
claimnumber = text.split('Claim Number: ')[1].split(' ')[0]
patientaccountno = text.split('Patient Account No.:')[1].split(' ')[0]
plannumber = text.split('Plan Number:')[1].split(' ')[0]
patientname = text.split('Patient Name: ')[1].split('Employee Name')[0]
employeename = text.split('Employee Name: ')[1].split('Relationship')[0]
realtionship = text.split('Relationship: ')[1].split('Planholder:')[0]
planholder = text.split('Planholder: ')[1]
print(claimnumber)
print(patientaccountno)
print(plannumber)
print(patientname)
print(employeename)
print(realtionship)
print(planholder)

36347F13200
120320346
00581682
Patient
ANDREW CASEY 
ANDREW CASEY 
SON

GOENGINEER, INC.


In [6]:
header = ['Line\nNo.', 'Submitted\nADA Codes/Description', 'Alt\nCode', 'Tooth\nNo.', 'Date of\nService', 'Submitted\nCharge', 'Considered\nCharge', 'Covered\nCharge', 'Deductible\nAmount', 'Coverage\nPercent', 'Benefit\nAmount']

In [7]:
head = []
for h in header:
    if 'Line' in h:
        head.append("LineNo")
    if 'Submitted\nADA' in h:
        head.append("SubmittedADACodesDescription")
    if 'Alt' in h:
        head.append("AltCode")
    if 'Tooth' in h:
        head.append("ToothNo")
    if 'Date of' in h:
        head.append("DateOfService")
    if 'Submitted\nCharge' in h:
        head.append("SubmittedCharge")
    if 'Considered' in h:
        head.append("ConsideredCharge")
    if 'Covered\nCharge' in h:
        head.append("CoveredCharge")
    if 'Deductible' in h:
        head.append("DeductibleAmount")
    if 'Coverage\nPercent' in h:
        head.append("CoveragePercent")
    if 'Benefit\nAmount' in h:
        head.append("BenefitAmount")
head      
        
        

['LineNo',
 'SubmittedADACodesDescription',
 'AltCode',
 'ToothNo',
 'DateOfService',
 'SubmittedCharge',
 'ConsideredCharge',
 'CoveredCharge',
 'DeductibleAmount',
 'CoveragePercent',
 'BenefitAmount']

In [8]:
import tabula
import pandas as pd
tabula_dfs = tabula.read_pdf(file_path,guess=False,pages=1,stream=True,encoding="utf-8",area=(141, 43, 176, 217),multiple_tables=True)

Got stderr: Jun 04, 2023 2:41:19 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:19 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:19 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:19 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:19 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:19 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:19 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding



In [9]:
tabula_dfs[0]

,UT BCDO SPECIALTY DENTAL SERVI
0,1275 E FORT UNION BLVD STE 100
1,"MIDVALE, UT 84047-1890"


In [10]:
def process_tabula_address(tabula_df: pd.DataFrame) -> str:
    col_name: str = tabula_df.columns[0]
    item: str = f"{col_name} {' '.join(tabula_df[col_name].to_list())}"
    return item

In [11]:
payee_address = process_tabula_address(tabula_dfs[0])

In [12]:
from Utilities.pdf_utils  import *
def extract_address_details(address_text):
    try:
        address_list = usaddress.tag(address_text)[0]
    except usaddress.RepeatedLabelError as e:
            parsed_address = e.parsed_string
            print('parsed address is for payor is----------', parsed_address )
            address_list = get_address_list_when_exception(parsed_address)


    return {
        'Item': address_text,
        'ItemName': address_list.get('Recipient', ''),
        'AddressElements': get_address(address_text, address_list),
        'PlaceName': address_list.get('PlaceName', ''),
        'StateName': address_list.get('StateName', ''),
        'ZipCode': address_list.get('ZipCode', '')
    }

C:\Users\pytho\AppData\Roaming\Python\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [13]:
payee_add_details = extract_address_details(payee_address)

In [14]:
payee_add_details

{'Item': 'UT BCDO SPECIALTY DENTAL SERVI 1275 E FORT UNION BLVD STE 100 MIDVALE, UT  84047-1890',
 'ItemName': 'UT BCDO SPECIALTY DENTAL SERVI',
 'AddressElements': '1275 E FORT UNION BLVD STE 100',
 'PlaceName': 'MIDVALE',
 'StateName': 'UT',
 'ZipCode': '84047-1890'}

In [15]:
tabula_dfs = tabula.read_pdf(file_path,guess=False,pages=1,stream=True,encoding="utf-8",area=(25, 90, 53, 213),multiple_tables=True)

Got stderr: Jun 04, 2023 2:41:20 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:20 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:20 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:20 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:20 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:20 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:20 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding



In [16]:
payer_address = process_tabula_address(tabula_dfs[0])

In [17]:
payer_address

'PO BOX 981572 EL PASO, TX  79998-1572'

In [18]:
payer_add_details = extract_address_details(payer_address)

In [19]:
payer_add_details

{'Item': 'PO BOX 981572 EL PASO, TX  79998-1572',
 'ItemName': '',
 'AddressElements': 'PO BOX 981572',
 'PlaceName': 'EL PASO',
 'StateName': 'TX',
 'ZipCode': '79998-1572'}

In [20]:
other_info = tabula.read_pdf(file_path,guess=False,pages=1,stream=True,encoding="utf-8",area=(97, 288, 152, 570),multiple_tables=True)

Got stderr: Jun 04, 2023 2:41:21 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:21 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:21 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:21 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:21 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:21 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding
Jun 04, 2023 2:41:21 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont readEncoding



In [21]:
df = other_info[0]

In [22]:
df = df.T.reset_index()
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

In [23]:
dit = df.to_dict(orient='records')

In [24]:
dit = dit[0]

In [25]:
dit

{'Provider:': 'DON REES BOREN',
 'Date:': '05/12/2023',
 'Payee:': 'UT BCDO SPECIALTY DENTAL SER',
 'Check No.:': '187857316',
 'Payment Amount:': '$351.20'}

In [26]:
from PyPDF2 import PdfReader
texts = ' '
with open(file_path, 'rb') as file:
    # Create a PdfReader object
    pdf = PdfReader(file)

    # Extract text from each page
    for page in pdf.pages:
        text = page.extract_text()
        texts = texts + text + ' '
 

C:\Users\pytho\AppData\Roaming\Python\Python310\site-packages\PyPDF2\_cmap.py:142: PdfReadWarning: Advanced encoding /BuiltIn not implemented yet
  warnings.warn(


In [27]:
texts

" If you have any\nquestions contact:(800) 541-7846\nWWW.GUARDIANANYTIME.COM\nProvider: DON REES BOREN\nDate: 05/12/2023\nPROVIDER EXPLANATION OF BENEFITS - THIS IS NOT A BILL\nImportant!  Please examine this statement for accuracy.  Save this statement for tax purposes.GROUP PLAN ADMINISTRATORS\nPayee:\nCheck No.:UT BCDO SPECIALTY DENTAL SER\n187857316\nPayment Amount: $351.20\nExpedite cash flow with e-payments. Sign up today!\nGuardian has contracted with Change Healthcare, a leading provider of \nrevenue and payment cycle solutions, to deliver Electronic Funds Transfer \n(EFT) services! Sign-up today by going to www.changehealthcare.com/support\n/customer-resources/enrollment-services  for more information. Enrollment for \nthis service is offered to you at no additional cost and is available online or by \ncalling 1.866.777.0713 and selecting Option 1.\nSubmitted\nADA Codes/DescriptionTooth\nNo.Date of\nServiceSubmitted\nChargeCoverage\nPercentAlt\nCodeConsidered\nChargeCovered\nC

In [28]:
# claimnumber = '18472E05500'
claimnumber = '36347F13200'
remarks = ''
remarks = texts.split(f"Remarks for claim # {claimnumber}:")[1]

if '10 Hudson' in  remarks:
    remarks = remarks.split('10 Hudson')[0]
else:
    remarks = remarks.split('Comments:')[0]

In [29]:
remarks.replace('\n', ' ').strip()

'AN AETNA DENTAL ACCESS PARTICIPATING DENTIST HAS BEEN UTILIZED 6.  The dental plan covers bitewing radiographic images only once in any 12 consecutive month period, up to a maximum of four films or a set of 7-8 vertical films, in one visit. Benefits are based on the use of a Preferred Contracted Dentist. You have used     $0.00 of your personal Maximum Rollover Account.   $700.00 remains in your account'

In [30]:
benefit = texts.split(f"Remarks for claim # {claimnumber}:")[0]
if 'Remarks for claim' in benefit:
    benefit = benefit.split('Remarks for claim')[1]
benefit

" # 36343F13200:\nAN AETNA DENTAL ACCESS PARTICIPATING DENTIST HAS BEEN UTILIZED\n4.  The treatment plan does not meet the clinical criteria (which is established based on the patient's\nage) for coverage of general anesthesia or sedation.\nBenefits are based on the use of a Preferred Contracted Dentist.\n10 Hudson Yards\n51-44\n119CHECK NO: 187857316\nPAY Three Hundred Fifty One & 20/100 Dollars****$351.20AMOUNT\nTO THE\nORDER OFUT BCDO SPECIALTY DENTAL SERVI\nBANK OF AMERICA\n150 WINDSOR STREET, HARTFORD, CT 06120CHECK DATE: 05/15/23\nVoid unless presented \nwithin 180 daysNew York, NY 10001\nADDTFDTATTFDFFAAATDTAADAFTTTTFTDAFAFAADFFFDDFDDFDFT DAFTTFFAAFATAD113 7.0940 AB 2.632ALL FOR ADC 840\nMIDVALE, UT  84047-18903 UT BCDO SPECIALTY DENTAL SERVI\n1275 E FORT UNION BLVD STE 100\nEL PASO, TX  79998-1572PO BOX 981572\nForwarding Service Requested\nC63224A544009110AC613758781CP627402800G\n    \nENV 113           9 OF 34 F P627402800G191410         If you have any\nquestions contact:(80

In [31]:
paidbyotherinsu = '$'+ benefit.split('PAID BY OTHER INSURANCE')[1].split('ADJUSTMENTS')[0].replace('\n','').split('$')[1]
paidbyotherinsu

'$0.00'

In [32]:
adj = '$'+ benefit.split('ADJUSTMENTS')[1].split('TOTAL BENEFIT PAID')[0].replace('\n','').split('$')[1]
adj

'$0.00'

In [33]:
totalbenefitpaid = '$'+ benefit.split('TOTAL BENEFIT PAID')[1].split('PATIENT')[0].replace('\n','').split('$')[1]
totalbenefitpaid

'$ 110.00'

In [34]:
patientrep = '$'+ benefit.split('PATIENT')[1].split('TOTALS\nTOTAL BENEFIT')[0].replace('\n','').split('$')[1].strip()
patientrep

'$30.00'

In [35]:
toalbenfitpayable = benefit.split('PAID BY OTHER INSURANCE')[0].split('\n')[-2].strip()
toalbenfitpayable

'$110.00'

In [36]:
higherallowable = '$'+ benefit.split('BENEFIT SUMMARY')[1].split('HIGHER ALLOWABLE')[0].replace('\n','').split('$')[1].strip()
higherallowable

'$110.00'